# LangChain 核心模块 Agent - Self ask with search

- Google 搜索对接
    - 第三方平台：https://serpapi.com
    - LangChain API 封装：`SerpAPIWrapper`
- LangChain Agent 模块使用
    - Agent 类型：`SELF_ASK_WITH_SEARCH`
    - Agent 实例化：`initialize_agent` 方法
 
https://python.langchain.com/docs/modules/agents/agent_types/

In [2]:
! pip install -U langchain

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple


In [5]:
pip install -U langchain-community

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
     ---------------------------------------- 0.0/2.2 MB ? eta -:--:--
      --------------------------------------- 0.0/2.2 MB 660.6 kB/s eta 0:00:04
      --------------------------------------- 0.0/2.2 MB 660.6 kB/s eta 0:00:04
      --------------------------------------- 0.0/2.2 MB 660.6 kB/s eta 0:00:04
     -- ------------------------------------- 0.1/2.2 MB 595.3 kB/s eta 0:00:04
     -- ------------------------------------- 0.1/2.2 MB 516.7 kB/s eta 0:00:05
     --- ------------------------------------ 0.2/2.2 MB 734.2 kB/s eta 0:00:03
     ---- ----------------------------------- 0.2/2.2 MB 724.0 kB/s eta 0:00:03
     ---- ----------------------------------- 0.2/2.2 MB 656.8 kB/s eta 0:00:04
     -------- ------------------------------- 0.5/2.2 MB 1.2 MB/s eta 0:00:02
     -------- ------------------------------- 0.5/2.2 MB 1.2 MB/s eta 0:00:02
     -------- ------------------------------- 0.5/2.2 MB 1.2 MB/s eta 0:

In [3]:
import os

# 更换为自己的 Serp API KEY
os.environ["SERPAPI_API_KEY"] = "cb155a954ea29b94f040f3154c39f47e4625c22f8d685967034f006e8fe05240"

In [6]:
from langchain_openai import OpenAI
from langchain.utilities import SerpAPIWrapper
from langchain.agents import initialize_agent, AgentType, Tool

llm = OpenAI(temperature=0, base_url="https://api.xiaoai.plus/v1", api_key="sk-20kg2ng944ZYhmdr87Db79E9323e48DaA1470c40B4B5439c" )

In [8]:
pip install google-search-results

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for google-search-results: filename=google_search_results-2.4.2-py3-none-any.whl size=32081 sha256=3e5361d1b8ae8ccf83fd6f35a9193f5c3898f6413c7d87abedf90304e6c795f5
  Stored in directory: c:\users\zhaomi3\appdata\local\pip\cache\wheels\e9\5f\b0\a373760764b227a31bd7fd12df8094664b68f31a9470c5c32e
Successfully built google-search-results
Note: you may need to restart the kernel to use updated packages.


In [9]:
# 实例化查询工具
search = SerpAPIWrapper()
tools = [
    Tool(
        name="Intermediate Answer",
        func=search.run,
        description="useful for when you need to ask with search",
    )
]

In [10]:
# 实例化 SELF_ASK_WITH_SEARCH Agent
self_ask_with_search = initialize_agent(
    tools, llm, agent=AgentType.SELF_ASK_WITH_SEARCH, verbose=True
)

D:\DevSoftware\miniconda3\envs\langchain\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.3.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(


In [11]:
# 实际运行 Agent，查询问题（正确）
self_ask_with_search.invoke(
    "成都举办的大运会是第几届大运会？2023年大运会举办地在哪里？"
)



> Entering new AgentExecutor chain...
 Yes
Follow up: What does "第几届" mean?
Intermediate answer: "第几届" means "which edition" or "which session". 
Follow up: In which year will the 2023 Universiade be held?
Intermediate answer: The 2023 Universiade will be held in 2023.
Follow up: Where will the 2023 Universiade be held?
Intermediate answer: The 2023 Universiade will be held in Chengdu, China.
So the final answer is: 成都大运会是第31届大运会，2023年大运会举办地在中国成都。

> Finished chain.


{'input': '成都举办的大运会是第几届大运会？2023年大运会举办地在哪里？',
 'output': '成都大运会是第31届大运会，2023年大运会举办地在中国成都。'}

In [12]:
# 实际运行 Agent，查询问题（有歧义）
self_ask_with_search.invoke(
    "2023年大运会举办地在哪里？成都举办的大运会是第几届大运会？"
)



> Entering new AgentExecutor chain...
 Yes.
Follow up: 哪里将举办2023年大运会？
Intermediate answer: 2023年大运会将在成都举办。
Follow up: 成都举办的大运会是第几届大运会？
Intermediate answer: 成都举办的大运会是第31届大运会。
So the final answer is: 第31届大运会

> Finished chain.


{'input': '2023年大运会举办地在哪里？成都举办的大运会是第几届大运会？', 'output': '第31届大运会'}

In [13]:
# Reason-only 答案（错误）
print(llm.invoke("成都举办的大运会是第几届大运会？"))



成都举办的大运会是第31届大运会。


In [14]:
# Reason-only 答案（错误）
print(llm.invoke("2023年大运会举办地在哪里？"))



目前还没有公布2023年大运会的举办地。根据国际大学生体育联合会的规定，大运会举办地需要提前三年确定，也就是说最快会在2020年确定。各国学生体育组织可以向国际大学生体育联合会提交申办意向，经过评估和投票后决定举办地。因此，2023年大运会举办地目前还未确定。


#### 使用 GPT-4 作为大语言模型实现更优的 ReAct 范式

In [16]:
from langchain_openai import ChatOpenAI

chat_model = ChatOpenAI(model="gpt-4o", temperature=0)

In [17]:
self_ask_with_search_chat = initialize_agent(
    tools, chat_model, agent=AgentType.SELF_ASK_WITH_SEARCH, verbose=True
)

In [18]:
# GPT-4 based ReAct 答案（正确）
self_ask_with_search_chat.invoke(
     "成都举办的大运会是第几届大运会？2023年大运会举办地在哪里？"
)



> Entering new AgentExecutor chain...
Yes.

Follow up: 成都举办的大运会是哪一年？
Intermediate answer: 大运会所有必选竞赛项目全部是奥运项目，竞赛管理由国际单项体育组织按照奥运标准实施，是规模仅次于奥运会的世界综合性运动会，素有“小奥运会”之称。 2023年7月28日至8月8日，第31届世界大学生夏季运动会在四川成都举办。
Intermediate answer: 2023年7月28日至8月8日，第31届世界大学生夏季运动会在四川成都举办。

So the final answer is: 第31届

Follow up: 2023年大运会举办地在哪里？
Intermediate answer: 2018年3月5日正式宣布普莱西德湖成为2023年冬季大运会的主办城市，国际大体联官员称这是大运会申办历史上最强有力的一次。 由于2021年冬季世界大学生运动会因2019冠状病毒病疫情而取消，将大运会会旗正式移交给普莱西德湖的仪式改在2022年1月于都灵举行。
Intermediate answer: 2023年冬季大运会的举办地是普莱西德湖。

So the final answer is: 普莱西德湖

> Finished chain.


{'input': '成都举办的大运会是第几届大运会？2023年大运会举办地在哪里？', 'output': '普莱西德湖'}